In [207]:
from aipython.stripsProblem import STRIPS_domain, Planning_problem
from aipython.stripsForwardPlanner import Forward_STRIPS
from aipython.searchMPP import SearcherMPP
from lab02.aipython.stripsForwardPlanner import State


In [208]:
from lab02.aipython.stripsProblem import Strips

# Define the STRIPS domain
delivery_domain = STRIPS_domain(
    {
        'RLoc': {'cs', 'off', 'lab', 'mr'},  # Possible robot locations
        'RHC': {False, True},  # Robot has cargo?
        'SWC': {False, True},  # Warehouse has cargo?
        'MW': {False, True},   # Robot has maintenance need?
        'RHM': {False, True}   # Robot has maintenance mode?
    },
    {  # Set of STRIPS actions
        Strips('mc_cs', {'RLoc': 'cs'}, {'RLoc': 'off'}),
        Strips('mc_off', {'RLoc': 'off'}, {'RLoc': 'lab'}),
        Strips('mc_lab', {'RLoc': 'lab'}, {'RLoc': 'mr'}),
        Strips('mc_mr', {'RLoc': 'mr'}, {'RLoc': 'cs'}),
        Strips('mcc_cs', {'RLoc': 'cs'}, {'RLoc': 'mr'}),
        Strips('mcc_off', {'RLoc': 'off'}, {'RLoc': 'cs'}),
        Strips('mcc_lab', {'RLoc': 'lab'}, {'RLoc': 'off'}),
        Strips('mcc_mr', {'RLoc': 'mr'}, {'RLoc': 'lab'}),
        Strips('puc', {'RLoc': 'cs', 'RHC': False}, {'RHC': True}),
        Strips('dc', {'RLoc': 'off', 'RHC': True}, {'RHC': False, 'SWC': False}),
        Strips('pum', {'RLoc': 'mr', 'MW': True}, {'RHM': True, 'MW': False}),
        Strips('dm', {'RLoc': 'off', 'RHM': True}, {'RHM': False})
    }
)


def heuristic(state, goal):
    """
    Heuristic function to estimate the cost to reach the goal state from the given state.
    """
    # Calculate the number of unsatisfied goals
    unsatisfied_goals = 0
    for key, value in goal.items():
        if state[key] != value:
            unsatisfied_goals += 1
    return unsatisfied_goals

def format_path(path):
    """
    Format the path for readability.
    """
    formatted_steps = []
    for i, step in enumerate(path.steps):
        state = step.state
        action = step.action
        if i == 0:
            formatted_steps.append(f"Initial State: {state}")
        else:
            formatted_steps.append(f"Step {i}: Perform action '{action}'")
            formatted_steps.append(f"Resulting State: {state}")
    return "\n".join(formatted_steps)


## Problem 1

In [209]:
# Define the initial state
initial_state = {'RLoc': 'lab', 'MW': True, 'SWC': True, 'RHC': False, 'RHM': False}

# Define the goal state
goal_state = {'SWC': False, 'MW': False, 'RHM': False}

# Create a State object with the initial state
initial_state_obj = State(initial_state)
goal_state_obj = State(goal_state)

# Calculate the heuristic value using the heuristic function and initial state
# heuristic_value = heuristic(initial_state_obj, goal_state_obj)

# Define the Planning problem
problem_1 = Planning_problem(delivery_domain, initial_state, goal_state)

# Instantiate the Forward_STRIPS class with the planning problem and heuristic function
forward_strips = Forward_STRIPS(problem_1)


# Use A* search algorithm to find a solution
solution = SearcherMPP(forward_strips).search()

# Print the solution
print(solution)

Solution: {'RLoc': 'lab', 'MW': True, 'SWC': True, 'RHC': False, 'RHM': False}
   --mc_lab--> {'RLoc': 'mr', 'MW': True, 'SWC': True, 'RHC': False, 'RHM': False}
   --pum--> {'RLoc': 'mr', 'MW': False, 'SWC': True, 'RHC': False, 'RHM': True}
   --mc_mr--> {'RLoc': 'cs', 'MW': False, 'SWC': True, 'RHC': False, 'RHM': True}
   --puc--> {'RLoc': 'cs', 'MW': False, 'SWC': True, 'RHC': True, 'RHM': True}
   --mc_cs--> {'RLoc': 'off', 'MW': False, 'SWC': True, 'RHC': True, 'RHM': True}
   --dm--> {'RLoc': 'off', 'MW': False, 'SWC': True, 'RHC': True, 'RHM': False}
   --dc--> {'RLoc': 'off', 'MW': False, 'SWC': False, 'RHC': False, 'RHM': False} (cost: 7)
 29 paths have been expanded and 16 paths remain in the frontier
{'RLoc': 'lab', 'MW': True, 'SWC': True, 'RHC': False, 'RHM': False}
   --mc_lab--> {'RLoc': 'mr', 'MW': True, 'SWC': True, 'RHC': False, 'RHM': False}
   --pum--> {'RLoc': 'mr', 'MW': False, 'SWC': True, 'RHC': False, 'RHM': True}
   --mc_mr--> {'RLoc': 'cs', 'MW': False, 'SWC

## Problem 2

In [210]:
initial_state_2 = { 'RLoc': 'lab', 'RHC': True, 'SWC': False, 'MW': True, 'RHM': False }

goal_state_2 = {'SWC': False, 'MW': False, 'RHM': False}

# Create a State object with the initial state
initial_state_obj_2 = State(initial_state)
goal_state_obj_2 = State(goal_state)

# Define the Planning problem
problem_2 = Planning_problem(delivery_domain, initial_state_2, goal_state_2)

# Solve the problem using the forward planner
forward_strips_2 = Forward_STRIPS(problem_2, heur=heuristic)
solution_2 = SearcherMPP(forward_strips_2).search()

    

Solution: {'RLoc': 'lab', 'RHC': True, 'SWC': False, 'MW': True, 'RHM': False}
   --mc_lab--> {'RLoc': 'mr', 'RHC': True, 'SWC': False, 'MW': True, 'RHM': False}
   --pum--> {'RLoc': 'mr', 'RHC': True, 'SWC': False, 'MW': False, 'RHM': True}
   --mc_mr--> {'RLoc': 'cs', 'RHC': True, 'SWC': False, 'MW': False, 'RHM': True}
   --mc_cs--> {'RLoc': 'off', 'RHC': True, 'SWC': False, 'MW': False, 'RHM': True}
   --dm--> {'RLoc': 'off', 'RHC': True, 'SWC': False, 'MW': False, 'RHM': False} (cost: 5)
 12 paths have been expanded and 11 paths remain in the frontier


## Problem 3

In [211]:
# Define the initial state and goal state
initial_state_3 = { 'RLoc': 'lab', 'RHC': True, 'SWC': True, 'MW': True, 'RHM': True }


goal_state = {'SWC': False, 'MW': False, 'RHM': False}

# Create a State object with the initial state
initial_state_obj_3 = State(initial_state)
goal_state_obj_3 = State(goal_state)

# Define the Planning problem
problem_3 = Planning_problem(delivery_domain, initial_state_3, goal_state)

# Solve the problem
forward_strips_3 = Forward_STRIPS(problem_3)
solution_3 = SearcherMPP(forward_strips_3).search()


Solution: {'RLoc': 'lab', 'RHC': True, 'SWC': True, 'MW': True, 'RHM': True}
   --mc_lab--> {'RLoc': 'mr', 'RHC': True, 'SWC': True, 'MW': True, 'RHM': True}
   --pum--> {'RLoc': 'mr', 'RHC': True, 'SWC': True, 'MW': False, 'RHM': True}
   --mc_mr--> {'RLoc': 'cs', 'RHC': True, 'SWC': True, 'MW': False, 'RHM': True}
   --mc_cs--> {'RLoc': 'off', 'RHC': True, 'SWC': True, 'MW': False, 'RHM': True}
   --dm--> {'RLoc': 'off', 'RHC': True, 'SWC': True, 'MW': False, 'RHM': False}
   --dc--> {'RLoc': 'off', 'RHC': False, 'SWC': False, 'MW': False, 'RHM': False} (cost: 6)
 28 paths have been expanded and 19 paths remain in the frontier
